# This script process the optimization1_quadtree results 

1. Compile the optimization1_quadtree with:
```
make bin/o1qtree
```
2. Run the benchmarking script
```
cd scripts
python3 run_par_optim1_qtree.py
```
3. The output that we will process is `o1_qtree_<hostname>.txt` that should be already saved in the `Results` folder.

In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the file with baseline results exists
o1_file = f'o1_qtree_{hostname}.txt'
# if it is not in Results
if not os.path.exists(o1_file):
    # is the file already in the directory?
    assert os.path.exists(os.path.join(hostname, o1_file)), f'File {o1_file} not found: something went wrong with the baseline benchmark.'
# if it is in Results
else:
    # copy the file to the directory
    assert os.system(f'mv {o1_file} {hostname}/') == 0, f'Failed to move {o1_file} to {hostname}/'
# rename o1_file
o1_file = os.path.join(hostname, o1_file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean

experiment ={}

# get the number of physical cores
nprocs = int(os.popen("lscpu | grep 'Core(s) per socket' | awk '{print $4}'").read().strip())
# number of threads used in the execution
num_threads = np.insert(np.linspace(2, nprocs, nprocs//2, dtype=int, endpoint=True), 0, 1)

with open(o1_file) as f:
    for line in f:
        tokens = line.split()
        if "Running:" in tokens:
            name=tokens[2].split("/")[3]
            nth=int(tokens[6])
            if name not in experiment:
                experiment[name]={}
        if 'Quadtree' in tokens:
            experiment[name][nth]=[float(tokens[5])]
        if "STAGE" in tokens:
            experiment[name][nth].append(float(tokens[5]))
        if 'Average:' in tokens:
            experiment[name][nth].append(float(tokens[1]))

#print(experiment)

results = {}

for i in experiment:
    results[i]={}
    for j in experiment[i]:
        results[i][j]={}
        results[i][j]['qtree']=experiment[i][j][0]
        results[i][j]['stage1']=mean(experiment[i][j][1:15:3])
        results[i][j]['stage2']=mean(experiment[i][j][2:15:3])
        results[i][j]['stage3']=mean(experiment[i][j][3:15:3])
        results[i][j]['owm']=experiment[i][j][16]
#print(results)
for i in experiment:
    print("Cloud {} has quadtree creation time = {:.2f}".format(i,mean(list(results[i][j]['qtree'] for j in [1,2,4,6,8]))))


In [ ]:
for i in experiment:
    print("Cloud {} has sequential OWM time = {:.2f}".format(i,results[i][1]['owm'] ))


In [ ]:
maxth = num_threads[-1]
for i in experiment:
    print("Cloud {} has parallel OWM with {} threads time(speedup) = {:.2f} ({:.1f}x)".format(i,maxth,results[i][maxth]['owm'],results[i][1]['owm']/results[i][maxth]['owm'] ))
    

# Comparing with baseline

In [ ]:
experiment_base ={}
with open(os.path.join(hostname, f'baseline_{hostname}.txt')) as f:
    for line in f:
        tokens = line.split()
        if "Running:" in tokens:
            name=tokens[2].split("/")[3]
            nth=int(tokens[6])
            if name not in experiment_base:
                experiment_base[name]={'seq':{}, 'par':{} }
        if "SEQUENTIAL" in tokens:
            status="seq"
        if 'CORES' in tokens:
            status="par"
        if 'Octree' in tokens:
            experiment_base[name][status][nth]=[float(tokens[5])]
        if "STAGE" in tokens:
            experiment_base[name][status][nth].append(float(tokens[5]))
        if 'Average:' in tokens:
            experiment_base[name][status][nth].append(float(tokens[1]))

#print(experiment_base)

results_base = {}

for i in experiment_base:
    results_base[i]={'seq':{}, 'par':{} }
    results_base[i]['seq']['octree']=experiment_base[i]['seq'][1][0]
    results_base[i]['seq']['stage1']=mean(experiment_base[i]['seq'][1][1:15:3])
    results_base[i]['seq']['stage2']=mean(experiment_base[i]['seq'][1][2:15:3])
    results_base[i]['seq']['stage3']=mean(experiment_base[i]['seq'][1][3:15:3])
    results_base[i]['seq']['owm']=experiment_base[i]['seq'][1][16]
    for j in experiment_base[i]['par']:
        results_base[i]['par'][j]={}
        results_base[i]['par'][j]['octree']=experiment_base[i]['par'][j][0]
        results_base[i]['par'][j]['stage1']=mean(experiment_base[i]['par'][j][1:15:3])
        results_base[i]['par'][j]['stage2']=mean(experiment_base[i]['par'][j][2:15:3])
        results_base[i]['par'][j]['stage3']=mean(experiment_base[i]['par'][j][3:15:3])
        results_base[i]['par'][j]['owm']=experiment_base[i]['par'][j][16]

octree_time = []
quadtree_time = []
for i in experiment_base:
    print("Cloud {} has Octree creation time = {:.2f}".format(i,mean(list(results_base[i]['par'][j]['octree'] for j in num_threads))))
    octree_time.append(mean(list(results_base[i]['par'][j]['octree'] for j in num_threads)))

for i,z in zip(experiment,range(len(experiment))):
    quadtree_time.append(mean(list(results[i][j]['qtree'] for j in num_threads)))
    print("Cloud {} has quadtree creation time = {:.2f} ({:.2f}x)".format(i,quadtree_time[z],octree_time[z]/quadtree_time[z]))

for i in experiment_base:
    print("Cloud {} has sequential OWM time = {:.2f}".format(i,results_base[i]['seq']['owm'] ))

for i in experiment:
    print("Cloud {} has sequential OWM time WITH opt1 = {:.2f} ({:.2f}x)".format(i,results[i][1]['owm'],results_base[i]['seq']['owm']/results[i][1]['owm'] ))     

for i in experiment_base:
    print("Cloud {} has parallel OWM with {} threads time(speedup) = {:.2f} ({:.1f}x)".format(i,maxth,results_base[i]['par'][maxth]['owm'],results_base[i]['seq']['owm']/results_base[i]['par'][maxth]['owm'] ))

for i in experiment:
    print("Cloud {} has parallel OWM with {} threads and opt1 time(speedup wrt base {} th) = {:.2f} ({:.1f}x)".format(i,maxth,maxth,results[i][maxth]['owm'],results_base[i]['par'][maxth]['owm']/results[i][maxth]['owm'] ))

for i in experiment:
    print("Cloud {} has parallel OWM with {} threads and opt1 time(speedup wrt base 1th) = {:.2f} ({:.1f}x)".format(i,maxth,results[i][maxth]['owm'],results_base[i]['seq']['owm']/results[i][maxth]['owm'] ))

In [ ]:
for i in experiment:
    for j in ['stage1','stage2','stage3']:
        print("Cloud {} stage {} time (percent) = {:.4f} ({:.2f}x)".format(i,j,results[i][1][j],results[i][1][j]/results[i][1]['owm']*100))


# Spedup for each cloud

In [ ]:
#Configuration variables
titlefs = 20
ylabelfs = 18
xlabelfs = 18
xticksfs = 16
yticksfs = 16
legendfs = 14
linew = 2
markers = 8

fig = plt.figure()

marks=['o-','x-','s-','v-','+-']

for (i,z) in zip(experiment,marks):
    print(np.array([results[i][1]['owm']/results[i][j]['owm'] for j in num_threads]))
    plt.plot(np.array(num_threads), np.array([results[i][1]['owm']/results[i][j]['owm'] for j in num_threads]), z, linewidth=linew, markersize=markers)
plt.plot(np.array(num_threads), np.array(num_threads), '-', linewidth=linew, markersize=markers)

sizes=[i for i in experiment]
sizes.append('Ideal')
plt.title(f'Speedup O1 {hostname.upper()}',  fontweight='bold', fontsize=titlefs)
plt.legend(sizes,loc='best', fontsize= legendfs)
plt.ylabel('Speedup', fontsize=ylabelfs)
plt.xlabel('Number of cores', fontsize=xlabelfs)
plt.xticks(num_threads,fontsize=xticksfs)
plt.yticks(fontsize=yticksfs)
plt.grid()

plt.show()

In [ ]:
pp = PdfPages(os.path.join(hostname, f'Speedup-opt1-quadtree-{hostname}.pdf'))
pp.savefig(fig)
pp.close()

# Save results in All_Optimizations.csv

In [ ]:
output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")
quadtree_t = []
for i,z in zip(results,range(len(experiment))):
    quadtree_t.append(mean(list(results[i][j]['qtree'] for j in num_threads)))
    print("Opt1-Quadtree; {}; {:.5f}; {:.5f};{};{}".format(i,quadtree_t[z],results[i][8]['owm'],0,0))
    f.write("Opt1-Quadtree;{};{:.5f};{:.5f};{};{}\n".format(i,quadtree_t[z],results[i][8]['owm'],0,0))

f.close()